In [4]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.integrate as integrate

from tqdm.notebook import trange, tqdm
import pickle


# Reconstruction

### TO DO:
- make reconstruction 
- compute fidelity
- compare different estimators (MLE, etc.)
- compare using different number of angles -- currently using 6 angles in $[0, \frac{5\pi}{6}]$
- compare using different binning for $X_\theta$

## Read data

In [21]:
data = np.load("measurements.npz")
thetas = data["thetas"]
measurements = data["measurements"]

with np.printoptions(precision=3, linewidth=150, edgeitems=10, suppress=True):
    print("      thetas:", thetas)
    print("measurements:", measurements)

# display metadata using bash command
!cat measurements_metadata.txt


      thetas: [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    ... 2.618 2.618 2.618 2.618 2.618 2.618 2.618 2.618 2.618 2.618]
measurements: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ... 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
state_type,cat
parity,odd
alpha,(-6+6j)
thetas,[0.0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966, 2.0943951023931953, 2.617993877991494]


## Background
The homodyne tomography experiment gives a set of data $(X_\theta, \theta)$  -- see [A. I. Lvovsky](../references/RevModPhys.81.299.pdf)

The data is generated in the [generateState notebook](generateState.ipynb) and save to the [data file](measurements.npz) and the [meta data file](measurements_metadata.txt)

In [22]:
def inverse_radon(X, theta, xmax=10, pmax=10, res=101):
    
    x = np.linspace(-xmax, xmax, res)
    p = np.linspace(-pmax, pmax, res)
    X, P = np.meshgrid(x, p)

    N = len(X)
    norm = 1/(2*np.pi**2*N)
    
    pass

### Reconstruction using Maximum Likelihood Estimator 
Based on [A. I. Lvovsky - section III.B](../references/RevModPhys.81.299.pdf)

The probability of measuring $j$ from a density matrix $\hat{\rho}$ is $\mathrm{P}_\rho(j) = \Tr\left[\hat{\Pi}_j\hat{\rho}\right]$

In [ ]:
def MLE_reconstruction(measurements, thetas, N=100, **kwargs):
    """
    Maximum Likelihood Estimation (MLE) for reconstructing the Wigner function.
    """
    
    xmax = kwargs.get('xmax', 8)
    xmin = kwargs.get('xmin', -xmax)
    
    pmax = kwargs.get('pmax', 8)
    pmin = kwargs.get('pmin', -pmax)
    
    x = np.linspace(xmin, xmax, N)
    p = np.linspace(pmin, pmax, N)
    X, P = np.meshgrid(x, p)

    # Initialize Wigner function
    W = np.zeros_like(X)

    # Perform MLE reconstruction
    for i, theta in enumerate(thetas):
        X_theta = (X * np.cos(theta) + P * np.sin(theta)) / np.sqrt(2)
        P_theta = (-X * np.sin(theta) + P * np.cos(theta)) / np.sqrt(2)
        W += measurements[i] * stats.norm.pdf(X_theta) * stats.norm.pdf(P_theta)

    return X, P, W